# Decomposing the Capacitated p-Median Problem
The capacitated p-median problem (CPMP) is a known and well-studied problem from the literature. Given
$n \in \mathbb{N}$ locations, the task is to select $p \in \mathbb{N}$ *median* locations with $p \leq n$ and to
assign each location to exactly one median. For any two locations $i,j \in [n]$, the distance between them is given
by $d_{ij} \in \mathbb{R}$. The distance between the locations and their assigned medians is minimized. Every
location $i \in [n]$ has a *demand* $q_i \in \mathbb{R}$ and a maximum *capacity* $Q_i \in \mathbb{R}$. For every
selected median, the sum of the demands of the locations assigned to it must not exceed its capacity.

The CPMP can be formulated as a MIP. Here is a classical compact formulation of problem:

$$
\begin{alignat*}{3}
z^{\star}_{IP}\,\,\,=\,\,\,\min{} &\sum_{i=1}^n \sum_{j=1}^n d_{ij} x_{ij} \hspace{-2em} &&&&\\
                     \text{s. t.} &\sum_{j=1}^n  x_{ij} &&= 1 \quad &\forall i &\in [n]\\
                                  &\sum_{i=1}^{n} q_i x_{ij} &&\leq Q_j y_j \quad &\forall j &\in [n] \\
                                  &\sum_{j=1}^n  y_j &&= p && \\
                                  &x_{ij} \in \{0,1\}, y_j \in \{0,1\} \hspace{-6em} &&\hspace{6em}\quad& \forall i,j &\in [n].
\end{alignat*}
$$

There are different approaches to solve the CPMP. As it has a structure, we can try to solve using a
Branch-Cut-and-Price approach. For that we want to use the `PyGCGOpt` interface to interact with GCG. We will consider three
use-cases: (1) The Automatic Mode, (2) Exploring different Decompositions, and (3) Building a custom Decomposition.

To follow along with this tutorial interactively, please download the Jupyter notebook from the [examples folder](https://github.com/scipopt/PyGCGOpt/tree/master/examples/cpmp).

## Reading in the Instance
The `PyGCGOpt` interface offers two methods to specify a problem. The first is to load the model from a standardized file format (e.g., `.lp` or `.mps`) that is supported by `SCIP` using `Model.readProb()`. Optionally, one can also read in a decomposition from a `.dec` file in the same manner. In this example, we will use the second method: Modeling a problem directly in Python.

Execute the following cell which includes a trivial test instance and function to load more instances from a custom `JSON` file format.

In [4]:
import json

def get_simple_instance():
    n = 5
    p = 2
    d = {0: {0: 0, 1: 25, 2: 46, 3: 43, 4: 30}, 1: {1: 0, 2: 22, 3: 20, 4: 22}, 2: {2: 0, 3: 22, 4: 40}, 3: {3: 0, 4: 22}, 4: {4: 0}}
    q = {0: 14, 1: 13, 2: 9, 3: 15, 4: 6}
    Q = {i: 33 for i in range(5)}
    return n, p, d, q, Q

def read_instance_json(path):
    with open(path) as f:
        instance = json.load(f)
    return instance["n"], instance["p"], instance["d"], instance["q"], instance["Q"]

n_locations, n_clusters, distances, demands, capacities = read_instance_json("instances/p550-01.json")

## Setting up the Model
Now, we want to build the model based on the above formulation. Please note that this part is *not* specific to GCG but is *almost* identical to how one would build the same model with `PySCIPOpt`. In fact, the only difference is that we import and instanciate `GCGModel` instead of `Model`. In technical terms, `GCGModel` is a subclass of `Model` and, therefore, you can use all methods of `Model` to build your problem.

In order to recreate the model multiple times during this example, we create a method that will return the model. The method also returns the different constraints added to the model grouped by type. This will be important later in use-case 3.

In [5]:
from pygcgopt import GCGModel, quicksum

def build_model(n_locations, n_clusters, distances, demands, capacities):
    m = GCGModel()

    m.printVersion()
    m.redirectOutput()

    m.setMinimize()

    x = {}
    y = {}

    for j in range(1, n_locations + 1):
        y[j] = m.addVar(f"y_{j}", vtype="B")
        for i in range(1, n_locations + 1):
            x[i, j] = m.addVar(f"x_{i}_{j}", vtype="B", obj=distances[i][j])

    # Hold different constraint types
    conss_assignment = []
    conss_capacity = []
    cons_pmedian = None

    # Create the assignment constraints
    for i in range(1, n_locations + 1):
        conss_assignment.append(
            m.addCons(quicksum(x[i, j] for j in range(1, n_locations + 1)) == 1)
        )

    # Create the capacity constraints
    for j in range(1, n_locations + 1):
        conss_capacity.append(
            m.addCons(quicksum(demands[i] * x[i, j] for i in range(1, n_locations + 1)) <= capacities[j] * y[j])
        )

    # Create the p-median constraint
    cons_pmedian = m.addCons(quicksum(y[j] for j in range(1, n_locations + 1)) == n_clusters)

    return m, conss_assignment, conss_capacity, cons_pmedian

## Use-Case 1: The Automatic Mode
With the model built, we can now simply call the `optimize()` function and let GCG do its magic.

In [6]:
m, *conss = build_model(n_locations, n_clusters, distances, demands, capacities)
m.optimize()

GCG version 3.5.0 [GitHash: b54569ac6-dirty]
Copyright (C) 2010-2021 Operations Research, RWTH Aachen University
                        Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)

SCIP version 8.0.0 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: SoPlex 6.0.0] [GitHash: 1af84b0617]
Copyright (C) 2002-2021 Konrad-Zuse-Zentrum fuer Informationstechnik Berlin (ZIB)


KeyError: 1

## Use-Case 2: Exploring different Decompositions
Above, we have seen GCG in its fully automatic mode. If we want to dig deeper, we can inspect the different decompositions that GCG detects. For that, we recreate the model and manually execute `presolve()` and `detect()` for the model. At this stage it is possible to list and visualize the found decompositions.

In [ ]:
m, *conss = build_model(n_locations, n_clusters, distances, demands, capacities)
m.presolve()
m.detect()

decomps = m.listDecompositions()

print("GCG found {} finnished decompositions.".format(len(decomps)))

### Inspecting Decompositions

The call to `listDecompositions()` returns a list of `PartialDecomposition` objects. We can print a decomposition using the Python `print()` function to get a summary or access different fields directly.

For a full overview of available methods, take a look at the online documentation for the `PartialDecomposition` class, or execute `help(d)` where `d` is a decomposition object.

In [ ]:
print(decomps)

d = decomps[2]

print(
    f"Decomp scores: {d.classicScore=:.04f}, {d.borderAreaScore=:.04f}, {d.maxWhiteScore=:.04f}, {d.maxForWhiteScore=:.04f}"
)

### Visualizing Decompositions
In addition, GCG can create graphical visualizations of decompositions. They can easily be displayed in a Jupyter nodebook like so:

In [ ]:
d

### Selecting Decompositions
After this investigation, we decide that we like this particular decomposition. The following code orders GCG to select our decomposition instead of an automatic one. Then, the optimization process is started.

In [ ]:
d.isSelected = True

m.optimize()

## Use-Case 3: Building a custom Decomposition
In the previous use-cases we run GCG with automatically detected decompositions. This is useful if we do not know the exact structure of a model but still want to exploit GCG's decomposition approach.

However, for our model we *do* know its structure. If you take another look at our `build_model()` method, you will notice that we store the created constraints in different variables based on their type. This is a typical approach when we want to specify a custom decomposition after building the model using the Python interface.

In the following code, we recreate our model and use the different constraint types fo select constraints for reformulation and constraints for the Dantzig-Wolfe master problem.

In [ ]:
m, conss_assignment, conss_capacity, cons_pmedian = build_model(
    n_locations, n_clusters, distances, demands, capacities
)

conss_master = conss_assignment + [cons_pmedian]
conss_reform = conss_capacity

pd = m.createPartialDecomposition()
pd.fixConssToMaster(conss_master)

for block, c in enumerate(conss_reform):
    pd.fixConsToBlock(c, block)

m.addPreexistingPartialDecomposition(pd)

m.optimize()

## Summary

With that, we have seen the most important features to use GCG as a solver through the Python interface. In a different example, we will take a look at how GCG can be extended using Python code.